In [1]:
# Declare the components with respective parameters
from data import PoloniexDataHandler
from strategy import BuyAndHoldStrategy
from portfolio import NaivePortfolio
from execution import SimulatedExecutionHandler
from queue import Queue
import time
from datetime import datetime, timedelta

markets = ['USDT_BTC', 'BTC_ETH', 'BTC_MAID', 'BTC_XMR', 'BTC_XRP']
events = Queue()
market_data = PoloniexDataHandler(
    markets[:1], events, start=datetime.now()-timedelta(days=2), end=datetime.now())
strategy = BuyAndHoldStrategy(market_data, events)
port = NaivePortfolio(market_data, events)
exchange = SimulatedExecutionHandler(events)

2017-03-01 00:08:45.465009 2017-03-03 00:08:45.465020
acquired 29864.0 total trades


In [2]:
continue_loop = True

while continue_loop:

    if market_data.continue_backtest == True:
        num_market_snapshots = float(len(list(market_data.market_data.values())[0]))
        num_backtested_snapshots = float(len(list(strategy.signals.values())[0]))
        backtest_progress = int(num_backtested_snapshots / num_market_snapshots * 100)
        print(backtest_progress)

        market_data.update_market_data()
    else:
        break
    
    # Handle the events
    while True:
        try:
            event = events.get(False)
        except:
            break
        else:
            if event is not None:
                if event.type == 'MARKET':
                    strategy.calculate_signals(event)
                    port.update_timeindex(event)

                elif event.type == 'SIGNAL':
                    port.update_signal(event)

                elif event.type == 'ORDER':
                    event.print_order()
                    exchange.execute_order(event)
                    
                elif event.type == 'FILL':
                    port.update_fill(event)

    # 10-Minute heartbeat
    
    #continue_loop = False

0
SHORT 0
Order: Symbol=USDT_BTC, Direction=SELL, Quantity=100, Price=1186.9855476029356
2
SHORT -100
4
LONG -100
Order: Symbol=USDT_BTC, Direction=BUY, Quantity=200, Price=1189.3841739442394
6
LONG 100
8
LONG 100
10
LONG 100
12
SHORT 100
Order: Symbol=USDT_BTC, Direction=SELL, Quantity=200, Price=1189.4734220755045
14
SHORT -100
16
SHORT -100
18
LONG -100
Order: Symbol=USDT_BTC, Direction=BUY, Quantity=200, Price=1191.2049965060653
20
LONG 100
22
LONG 100
24
LONG 100
26
LONG 100
28
LONG 100
30
LONG 100
32
LONG 100
34
LONG 100
36
LONG 100
38
LONG 100
40
LONG 100
42
SHORT 100
Order: Symbol=USDT_BTC, Direction=SELL, Quantity=200, Price=1224.4340312359082
44
SHORT -100
46
LONG -100
Order: Symbol=USDT_BTC, Direction=BUY, Quantity=200, Price=1229.4375790734339
48
LONG 100
51
LONG 100
53
LONG 100
55
SHORT 100
Order: Symbol=USDT_BTC, Direction=SELL, Quantity=200, Price=1225.866016600469
57
SHORT -100
59
SHORT -100
61
SHORT -100
63
LONG -100
Order: Symbol=USDT_BTC, Direction=BUY, Quantity=200,

In [9]:
port.current_holdings

{'USDT_BTC': -130839.20565837601,
 'cash': 230835.80565837605,
 'commission': 3.3999999999999995,
 'total': 230835.80565837605}

In [3]:
%matplotlib inline
from pylab import *
import pandas as pd
prices = market_data.market_data['USDT_BTC'].rate['mean']
rolling_mean = market_data.market_data['USDT_BTC'].rate['mean'].rolling(window=6).mean()
signals = 1*-pd.Series(strategy.signals['USDT_BTC'][1:], index=market_data.market_data['USDT_BTC'].index)
signals = signals.diff()
fig = plt.figure(figsize=(20,10))
    # Set the outer colour to white
ax1 = fig.add_subplot(211,  ylabel='Price in $')

# Plot the AAPL closing price overlaid with the moving averages
prices.plot(ax=ax1, color='r', lw=2.)
rolling_mean.plot(ax=ax1, lw=2.)

# Plot the "buy" trades against AAPL
ax1.plot(signals[signals==1.0].index, 
         prices[signals== 1.0],
         '^', markersize=10, color='m')

# Plot the "sell" trades against AAPL
ax1.plot(signals[signals == -1.0].index, 
         prices[signals == -1.0],
         'v', markersize=10, color='k')
    

# Plot the figure
fig.show()

TypeError: bad operand type for unary -: 'dict'